In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

/home/porya/anaconda3/envs/azureml_py310/lib/python3.10/site-packages/azureml/core/__init__.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [12]:
from azure.storage.blob import ContainerClient


account_name = "reviewthebook"
container_name = "bookfiles"
sas_token = "se=2026-12-01&sp=racwdl&sv=2022-11-02&sr=c&sig=c%2Bv7BFK3r7MYzM3MTDsFDdVr1EBh9eK7o%2BNJEFdudmk%3D"


# Make sure the SAS token starts with '?'
if not sas_token.startswith('?'):
   sas_token = '?' + sas_token


container_url = f"https://{account_name}.blob.core.windows.net/{container_name}{sas_token}"
container_client = ContainerClient.from_container_url(container_url)


prefix = "melb_data.csv/"
blobs_to_delete = container_client.list_blobs(name_starts_with=prefix)
for blob in blobs_to_delete:
   container_client.delete_blob(blob.name)
   print(f"Deleted: {blob.name}")

Deleted: melb_data.csv/MELBOURNE_HOUSE_PRICES_LESS.csv
Deleted: melb_data.csv/Melbourne_housing_FULL.csv
Deleted: melb_data.csv/melb_data.csv


In [ ]:
from azureml.core import Datastore

datastore_name = "reviewthebookdatastore"
datastore = Datastore.get(ws, datastore_name)
local_file = "/home/porya/Desktop/Azure_book/local_dataset/melb_data.csv"

datastore.upload_files(
    files=[local_file],      # list, even for one file
    target_path="",          # folder in datastore
    overwrite=True,
    show_progress=True,
)
#2- now upload and register the above uploaded format as dataset
from azureml.core import Dataset

dataset = Dataset.File.from_files(path=[(datastore, "melb_data.csv")])
dataset = dataset.register(
    workspace=ws,
    name="melb_data_csv_202601",
    description="melb_dataset for book chp4 test registered on 2026-01-03",
    create_new_version=True)

Uploading an estimated of 1 files
Uploading /home/porya/Desktop/Azure_book/local_dataset/melb_data.csv
Uploaded /home/porya/Desktop/Azure_book/local_dataset/melb_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [15]:
# how to get the registered dataset
from azureml.core import Dataset

ds = Dataset.get_by_name(ws, name="melb_data_csv_202601", version=1)
ds

{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"azureml://subscriptions/d7a578a6-ad8a-4139-a8e5-b11a9cc35c10/resourcegroups/book_rg/workspaces/book_ws/datastores/reviewthebookdatastore/paths/melb_data.csv\"\nmetadata:\n  infer_column_types: \"False\"\n",
  "registration": {
    "id": "adcecbc1-293f-4f6a-9658-2ae824852630",
    "name": "melb_data_csv_202601",
    "version": 1,
    "description": "melb_dataset for book chp4 test registered on 2026-01-03",
    "workspace": "Workspace.create(name='book_ws', subscription_id='d7a578a6-ad8a-4139-a8e5-b11a9cc35c10', resource_group='book_rg')"
  }
}

In [27]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()


from azureml.core import Dataset
ds = Dataset.get_by_name(ws , name="titanic", version=1)
ds


{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv\"\ntransformations:\n  - read_delimited:\n      path_column: Path\n      include_path_column: false\n      encoding: utf8\n      support_multi_line: false\n      delimiter: \",\"\n      empty_as_string: false\n      partition_size: 20971520\n      header: all_files_same_headers\n      infer_column_types: false\n  - convert_column_types:\n      - columns: Survived\n        column_type: boolean\n      - columns: Name\n        column_type: string\n      - columns: Pclass\n        column_type: int\n      - columns: Fare\n        column_type: float\n      - columns: Embarked\n        column_type: string\n      - columns: SibSp\n        column_type: int\n      - columns: Ticket\n        column_type: string\n      - columns: PassengerId\n        column_type: int\n      - columns: Cabin\n        column_type: string\n      - columns: Sex\n        column_typ

In [28]:
pandas_df = ds.to_pandas_dataframe()
pandas_df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [ ]:
#TabularDataset directly from datastore
from azureml.core import Datastore, Dataset

datastore = Datastore.get(ws, "reviewthebookdatastore")

tab_ds = Dataset.Tabular.from_delimited_files(
    path=(datastore, "melb_data.csv")   # (Datastore, relative path in container)
)

pandas_df = tab_ds.to_pandas_dataframe()
pandas_df.head()


{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,2016-09-03,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,2016-12-03,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,2016-02-04,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,2016-02-04,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,2017-03-04,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [9]:
# Download the dataset files to a local directory
download_path = ds.download(target_path='./local_dataset', overwrite=True)


{'infer_column_types': 'False', 'activity': 'download'}
{'infer_column_types': 'False', 'activity': 'download', 'activityApp': 'FileDataset'}


In [12]:
# download_path is a list of file paths
print(download_path)  # Shows list of file paths

# Read the first CSV file directly
import pandas as pd
df = pd.read_csv(download_path[0])  # Adjust index or find correct file if multiple
df.head()


['/home/porya/Desktop/Azure_book/local_dataset/melb_data.csv']


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [14]:
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,34857.000000,2.724700e+04,34856.000000,34856.000000,26640.000000,26631.000000,26129.000000,23047.000000,13742.00000,15551.000000,26881.000000,26881.000000,34854.000000
mean,3.031012,1.050173e+06,11.184929,3116.062859,3.084647,1.624798,1.728845,593.598993,160.25640,1965.289885,-37.810634,145.001851,7572.888306
std,0.969933,6.414671e+05,6.788892,109.023903,0.980690,0.724212,1.010771,3398.841946,401.26706,37.328178,0.090279,0.120169,4428.090313
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1196.000000,-38.190430,144.423790,83.000000
25%,2.000000,6.350000e+05,6.400000,3051.000000,2.000000,1.000000,1.000000,224.000000,102.00000,1940.000000,-37.862950,144.933500,4385.000000
50%,3.000000,8.700000e+05,10.300000,3103.000000,3.000000,2.000000,2.000000,521.000000,136.00000,1970.000000,-37.807600,145.007800,6763.000000
75%,4.000000,1.295000e+06,14.000000,3156.000000,4.000000,2.000000,2.000000,670.000000,188.00000,2000.000000,-37.754100,145.071900,10412.000000
max,16.000000,1.120000e+07,48.100000,3978.000000,30.000000,12.000000,26.000000,433014.000000,44515.00000,2106.000000,-37.390200,145.526350,21650.000000


In [17]:
#passing external data as a URL
from azureml.core import Dataset

path = "https://dprepdata.blob.core.windows.net/demo/Titanic.csv"
ds = Dataset.Tabular.from_delimited_files(path)
ds.to_pandas_dataframe().head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,False,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,True,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,True,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,True,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,False,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [ ]:
dbutils.widgets.dropdown(name='Reset', defaultValue='True', choices=['True', 'False'], label="Reset: Delete previous data")

# COMMAND ----------

# MAGIC %run ../../global-setup $project_name=rag_chatbot

# COMMAND ----------

# DBTITLE 1,Notebook Variables
library_folder = "{}/raw_documents".format(volume_file_path)

user_agent = "mlaction_book"

# COMMAND ----------

if bool(dbutils.widgets.get('Reset')):
  dbutils.fs.rm(library_folder, recurse=True)
  dbutils.fs.mkdirs(library_folder)

# COMMAND ----------

# DBTITLE 1,Helper Functions
import os
import requests

def load_file(file_uri, file_name, library_folder):
    
    # Create the local file path for saving the PDF
    local_file_path = os.path.join(library_folder, file_name)

    # Download the PDF using requests
    try:
        # Set the custom User-Agent header
        headers = {"User-Agent": user_agent}

        response = requests.get(file_uri, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Save the PDF to the local file
            with open(local_file_path, "wb") as pdf_file:
                pdf_file.write(response.content)
            print("PDF downloaded successfully.")
        else:
            print(f"Failed to download PDF. Status code: {response.status_code}")
    except requests.RequestException as e:
        print("Error occurred during the request:", e)


# COMMAND ----------

# MAGIC %md
# MAGIC ### Download documents

# COMMAND ----------

# we are getting our documents, you could directly upload it to the volumes using UI
pdfs = {
        '2312.14565.pdf': 'https://arxiv.org/pdf/2312.14565.pdf', #used for evaluator, created 20 questions from it and answers generated using Mixtral
        '2303.10130.pdf':'https://arxiv.org/pdf/2303.10130.pdf', #used for evaluator, created 20 questions from it and answers generated using Mixtral
        '2302.06476.pdf':'https://arxiv.org/pdf/2312.00506.pdf', 
        '2302.06476.pdf':'https://arxiv.org/pdf/2302.06476.pdf', 
        '2311.07071.pdf':'https://arxiv.org/pdf/2311.07071.pdf',
        '2304.07683.pdf':'https://arxiv.org/pdf/2304.07683.pdf'}

for pdf in pdfs.keys():
    load_file(pdfs[pdf], pdf, library_folder)

# COMMAND ----------

dbutils.fs.ls(library_folder)